In [1]:
from collections import deque
import sys
import pandas as pd
import time

from urllib.request import urlopen
from bs4 import  BeautifulSoup

from requests import get



In [2]:
print("Python version")
print (sys.version)

Python version
3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]


In [3]:
request_header = {
        "Request_from" : "BRIAN PALLANGYO",
        "Email" : "<pallangyobrian@yahoo.com>",
        "Purpose" : "Academic Research Study",
        "Research_title" : "Comparative study of page rank and hits algorithms for reciprocal link prediction in online social networks"
         }

In [4]:
mayocoo_base_url = "https://www.mayocoo.com/"
following = "/following"
followers = "/followers"

In [5]:
def mayocoo_user (user):
    mayocoo_link = mayocoo_base_url + user
    return mayocoo_link

def mayocoo_user_follow (user):
    mayocoo_follow_link = mayocoo_base_url + user + following
    return mayocoo_follow_link

def mayocoo_user_follower (user):
    mayocoo_followers_link = twitter_base_url + user + followers
    return mayocoo_followers_link

def url_to_following (url_string):
    mayocoo_following = url_string + following
    return mayocoo_following

def get_user_details(user_name_id):
    user_name_profile_link = mayocoo_user(user_name_id)
    user_name_following_link = mayocoo_user_follow(user_name_id)
    user_name_follower_link = mayocoo_user_follower(user_name_id)
    number_of_following
    number_of_followers
    number_of_posts
    location
    gender
    date_of_birth

Starting Crawling Procedure

In [6]:
# Creating Queue
queue = deque([])

In [7]:
# Store Visited Users
visited_user_list = []

In [8]:
final_adjacency_list = ""
final_node_list = ""

In [9]:
#enter SEED NODE
seed_node = "https://www.mayocoo.com/justice"

In [11]:
#Crawling and entering found users in the queue
def crawl(url):
    visited_user_list.append(url)
    global final_adjacency_list
    global final_node_list
    
    if len(queue) > 100000: #limit queue length to 100000nodes
        return
             
    mayocoo_link = url_to_following (url) #go to user following page
    try:
        html = urlopen(mayocoo_link)
    except:
        #RESUME 
        #get url from queue to visit
        next_url = queue.popleft()
        #Call the Crawl function
        crawl(next_url)
    
    try:
        soup = BeautifulSoup(html, 'html.parser')
    except:
        #RESUME 
        next_url = queue.popleft()
        crawl(next_url)
        
    result = soup.find_all('span', {'class' : 'user-popover'})
    
    #GET USER ATTRIBUTES
    source_node = soup.find('img', {'class': 'pointer timelineuserpropic'}).get("id")[12:]
    username = soup.find('a', {'class': 'pronameblack'}).get_text().strip()
    user_details = soup.find('ul', {'class': 'list-group'})
    
 
    #Initialize empty string for user details to overcome errors when no detail
    user_no_posts = " "
    user_gender = " "
    date_of_birth = " "
    user_work = " "
    studying = " "
    country = " "
    region = " "
    
    for user_detail in user_details.find_all('li'):
        if (user_detail.find_all('i', {'class': 'zmdi zmdi-edit zmdi-hc-stack-1x'})) != []: #NUmber of POSTS
            user_no_posts = user_details.find_all('li')[1].get_text().strip()[:-6]
        if (user_detail.find_all('i', {'class': 'zmdi zmdi-male-female zmdi-hc-stack-1x'})) != []: #GENDER
            user_gender = user_details.find_all('li')[3].get_text().strip()
        if (user_detail.find_all('i', {'class': 'zmdi zmdi-calendar zmdi-hc-stack-1x'})) != []: #DOB
            date_of_birth = user_detail.get_text().strip()
        if (user_detail.find_all('i', {'class': 'zmdi zmdi-case zmdi-hc-stack-1x'})) != []: #WORKING
            user_work = user_detail.get_text().strip()[11:]
        if (user_detail.find_all('i', {'class': 'zmdi zmdi-book zmdi-hc-stack-1x'})) != []: #studying
            studying = user_detail.get_text().strip()[12:]
        if (user_detail.find_all('i', {'class': 'zmdi zmdi-globe zmdi-hc-stack-1x'})) != []: #COUNTRY
            country = user_detail.get_text().strip()[27:].replace(',', '')
        if (user_detail.find_all('i', {'class': 'zmdi zmdi-pin zmdi-hc-stack-1x'})) != []: #REGION
            region = user_detail.get_text().strip()[11:].replace(',', '')
    
    
    #VISIT THE USER NODE
    current_node_attrib = source_node + "," + url + "," + username + "," + user_no_posts + "," + user_gender + "," + date_of_birth + "," + user_work + "," + studying + "," + country + "," + region + ", "
    #final_node_list = final_node_list + current_node_attrib 
    
    with open("mayocoo_nodeattrib.txt", "a+") as text_file:
        print(f"{current_node_attrib}", file=text_file)
        
    current_adjacency_list = source_node

    
    #Get user details
    
    ''' print(link_text.get("data-id"))
        print(link_text.a.get("href"))
        print(link_text.a.get_text())
        print("\t")
    '''
    
    for link_text in result:
        flag = 0
        current_adjacency_list = current_adjacency_list + "," + str(link_text.get("data-id"))
        
        #Get followed user url only
        user_url = link_text.a.get("href")
        
        #print ("Following USER_URL : ", user_url)
        
        #Check if user url exist in the queue
        for j in queue:
            if j == user_url:
                flag = 1
                break #if user url exist in queue then exit
                
        #if user url is not in the queue then proceed with execution len(queue)<100000
        if flag == 0:
            if len(queue) > 100000:
                return
            if (visited_user_list.count(user_url)) == 0:
                queue.append(user_url)
        
    #final_adjacency_list = final_adjacency_list + "\n" + current_adjacency_list
    with open("mayocoo_adjlist.txt", "a+") as text_file:
        print(f"{current_adjacency_list}", file=text_file)
    
    
    
    #get url from queue to visit
    next_url = queue.popleft()
    #print ("NEXT_URL : ", next_url)
    
    #Call the Crawl function
    crawl(next_url)
        
crawl(seed_node)
        
        
    

In [12]:
print (final_adjacency_list)
print (final_node_list)

In [13]:
print ("Number of visited :  %d" % len(visited_user_list))
with open("visited_user_list.txt", "a+") as visited_text:
    for each_list in visited_user_list:
        print (f"{each_list}", file=visited_text)

Number of visited :  2
